# Treinamento de Fraudes com Transformers

*Autor: Prof. Dr. Adriano Leal

Data de Criação: 13 de Abril de 2024

---

## Descrição
Este notebook Jupyter foi desenvolvido para automatizar .

## Not Working yet
---

## Log de Modificações

- **13/04/2024 - Criação inicial do notebook e implementação das funções básicas de importação por Adriano Leal.
- **[Data da Modificação]** - Adicionado suporte para logs de erro detalhados e tratamento de exceções por [Nome do Colaborador ou Seu Nome].
- **[Data da Modificação]** - Melhoria na interface de seleção de arquivos e otimização de performance por [Nome do Colaborador ou Seu Nome].

---

## Notas Adicionais
Este notebook é parte do projeto [Nome do Projeto] destinado a melhorar a gestão e análise de dados relacionados a deslizamentos.



!pip install -U  "ray[tune]" tensorboardX

In [1]:
import os
import nvidia.cudnn
pasta=nvidia.cudnn.__file__
print(pasta)

/opt/conda/lib/python3.11/site-packages/nvidia/cudnn/__init__.py


In [2]:

os.environ["TUNE_DISABLE_STRICT_METRIC_CHECKING"] = "1"
os.environ["TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S"] = "2"
#os.environ["LD_LIBRARY_PATH"]="/usr/local/cuda-12.4/targets/x86_64-linux/include:/opt/conda/lib/python3.11/site-packages/tensorrt_bindings/:/opt/conda/lib/python3.11/site-packages/tensorrt_libs:/usr/lib/python3.10/dist-packages/tensorrt:"+os.environ["LD_LIBRARY_PATH"]
os.environ['PYAV_LOGGING']='off'
import sys
sys.setrecursionlimit(10000)  # Increase cautiously



In [3]:
!dpkg -l | grep TensorRT  # For Debian/Ubuntu


ii  graphsurgeon-tf                                    8.6.1.6-1+cuda12.0                      amd64        GraphSurgeon for TensorRT package
ii  libnvinfer-bin                                     10.0.1.6-1+cuda12.4                     amd64        TensorRT binaries
ii  libnvinfer-dev                                     10.0.1.6-1+cuda12.4                     amd64        TensorRT development libraries
ii  libnvinfer-dispatch-dev                            10.0.1.6-1+cuda12.4                     amd64        TensorRT development dispatch runtime libraries
ii  libnvinfer-dispatch10                              10.0.1.6-1+cuda12.4                     amd64        TensorRT dispatch runtime library
ii  libnvinfer-headers-dev                             10.0.1.6-1+cuda12.4                     amd64        TensorRT development headers
ii  libnvinfer-headers-plugin-dev                      10.0.1.6-1+cuda12.4                     amd64        TensorRT plugin headers
ii  libnvinfer-lean-dev   

In [4]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("Number of GPUs Available:", len(tf.config.list_physical_devices('GPU')))

# Assuming TensorRT Python API is installed (`tensorrt` package)
import tensorrt as trt
print("TensorRT version:", trt.__version__)
print("TensorRT version:", trt.__file__)

2024-06-20 21:47:11.607414: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-20 21:47:13.030300: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow version: 2.16.1
Number of GPUs Available: 1
TensorRT version: 10.1.0
TensorRT version: /opt/conda/lib/python3.11/site-packages/tensorrt/__init__.py


2024-06-20 21:47:14.611808: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 21:47:14.634402: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-20 21:47:14.634457: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [5]:
!echo $PATH

/opt/conda/bin:/root/gems/bin:/opt/mssql-tools18/bin:/usr/local/cuda-12.4/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/conda/bin:/opt/conda/lib/python3.11/site-packages/nvidia/cudnn:/datatensor:/usr/lib/x86_64-linux-gnu:/opt/conda/lib/python3.11/site-packages/torch/lib:/opt/conda/condabin:/opt/conda/lib:/opt/conda/bin:/opt/conda/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/root/mambaforge/lib:/opt/conda/lib/python3.11/site-packages/tensorflow/include/third_party/gpus/cuda/include/:/opt/conda/lib/python3.11/site-packages/nvidia/cuda_runtime/lib:/opt/conda/lib:/usr/lib/x86_64-linux-gnu/:/usr/local/cuda-12.2/compat/:/opt/conda/lib/:/opt/conda/lib:/opt/conda/lib/python3.11/site-packages/nvidia/cuda_runtime/lib:/root/.julia/juliaup/julia-1.10.4+0.x64.linux.gnu/bin/:/opt/conda/lib/python3.11/site-packages/nvidia/cuda_runtime/lib/:/opt/mssql-tools18/bin:/usr/local/cuda-12

In [6]:
!echo $LD_LIBRARY_PATH
!echo $GEM_HOME
!echo $HOME

/usr/local/cuda-12.4/lib64:/usr/local/cuda-12.4/targets/x86_64-linux/include:/usr/local/cuda/include/:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/opt/conda/lib/python3.11/site-packages/tensorrt_bindings/:/opt/conda/lib/python3.11/site-packages/tensorrt_libs:/usr/lib/python3.10/dist-packages/tensorrt:/usr/local/cuda-12.4/lib64:/usr/local/cuda-12.4/targets/x86_64-linux/lib/:/opt/conda/lib:/opt/conda/lib/python3.11/site-packages/nvidia/cudnn/lib:/usr/lib/x86_64-linux-gnu:/usr/local/cuda-12.4/include:/usr/local/cuda-12.4:/usr/local/cuda-12.4/targets/x86_64-linux/lib/:/lib:/lib/:/usr/local/cuda/extras/CUPTI/lib64:/usr/lib/x86_64-linux-gnu/:/usr/lib/x86_64-linux-gnu/odbc/:/usr/lib/x86_64-linux-gnu/:/opt/conda/lib/python3.11/site-packages/nvidia/cuda_runtime/lib:/opt/conda/lib:/usr/lib/x86_64-linux-gnu/:/usr/local/cuda-12.2/compat/:/opt/conda/lib/:/opt/conda/lib:/opt/conda/lib/python3.11/site-packages/nvidia/cuda_runtime/lib:/opt/conda/lib/python3.11/site-packages/nvidia/cublas/include:/op

In [7]:
import tensorrt
print(tensorrt.__version__)
#assert tensorrt.Builder(tensorrt.Logger())

10.1.0


In [8]:
# Importações necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import datasets
import ray
from ray import tune
from ray.train import RunConfig
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler


In [9]:
def load_data(filepath, target_position=-1, date_column=None, encoding=None):
    """
    Carrega dados de um arquivo, tentando várias codificações em caso de falha.
    
    Args:
    - filepath: Caminho para o arquivo de dados.
    - target_position: Posição da coluna target, padrão é -1 (última coluna).
    - date_column: Nome da coluna de data para ordenar os dados, se aplicável.
    - encoding: Codificação do arquivo de dados. Padrão é None, tentará 'utf-8', 'ISO-8859-1' e 'windows-1252'.
    
    Returns:
    - X_train, X_test, y_train, y_test: Dados divididos em conjuntos de treino e teste.
    """
    # Tentativas de codificações
    encodings = ['utf-8', 'ISO-8859-1', 'windows-1252'] if encoding is None else [encoding]

    for enc in encodings:
        try:
            if filepath.lower().endswith('.csv'):
                df = pd.read_csv(filepath, encoding=enc)
            elif filepath.lower().endswith('.xlsx') or filepath.lower().endswith('.xls'):
                try:
                    df = pd.read_excel(filepath, engine='openpyxl')
                except ValueError:
                    df = pd.read_excel(filepath, engine='xlrd')
            else:
                raise ValueError("Formato de arquivo não suportado!")

            # Verificar se a coluna de data existe e ordenar os dados por ela
            if date_column and date_column in df.columns:
                df[date_column] = pd.to_datetime(df[date_column])
                df.sort_values(date_column, inplace=True)

            # Separação das features e target
            X = df.iloc[:, :target_position]  # Todas as colunas exceto a última
            y = df.iloc[:, target_position]   # A última coluna é considerada como target

            # Divisão dos dados em treino e teste
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
            return X_train, X_test, y_train, y_test

        except UnicodeDecodeError:
            print(f"Falha ao decodificar com {enc}, tentando outra codificação...")
        except Exception as e:
            print(f"Erro ao carregar o arquivo com codificação {enc}: {e}")
            continue  # Tenta a próxima codificação

    print("Não foi possível carregar o arquivo com as codificações tentadas.")
    return None, None, None, None

# Exemplo de uso da função
#X_train, X_test, y_train, y_test = load_data('your_data_file.xlsx', date_column='optional_date_column')




# Função para configurar e treinar modelos ORIGINAL
# Função para configurar e treinar modelos
def train_and_evaluate(model_name, config):
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

        # Tokenização
        train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=config.get("max_length", 512))
        test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=config.get("max_length", 512))

        # Criação de datasets
        train_dataset = datasets.Dataset.from_dict({'input_ids': train_encodings['input_ids'], 'labels': y_train.tolist()})
        test_dataset = datasets.Dataset.from_dict({'input_ids': test_encodings['input_ids'], 'labels': y_test.tolist()})

        # Configurações de treinamento
        training_args = TrainingArguments(
            output_dir=f'./results/{model_name}',
            num_train_epochs=config["num_epochs"],
            learning_rate=config['lr'],
            per_device_train_batch_size=config['batch_size'],
            evaluation_strategy='epoch'
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=test_dataset,
            compute_metrics=lambda p: {'accuracy': accuracy_score(p.label_ids, np.argmax(p.predictions, axis=1))}
        )

        # Treinamento e avaliação
        trainer.train()
        eval_results = trainer.evaluate()

        # Reportar a perda de avaliação para o Ray Tune
        tune.CLIReporter(metric_columns=[loss=eval_results['eval_loss'], accuracy=eval_results['eval_metrics']['accuracy']])
        #reporter = CLIReporter(metric_columns=["accuracy", "loss"])  # Adjust columns based on your metrics

        # Matriz de confusão
        preds_output = trainer.predict(test_dataset)
        preds = np.argmax(preds_output.predictions, axis=-1)
        cm = confusion_matrix(y_test, preds)
        plt.figure(figsize=(7,5))
        sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'])
        plt.xlabel('Predicted labels')
        plt.ylabel('True labels')
        plt.title(f'Confusion Matrix for {model_name}')
        plt.show()

        print(f"Model: {model_name}, Accuracy: {accuracy_score(y_test, preds)}")
        return eval_results['eval_loss']

    except Exception as e:
        print(f"Error training model {model_name}: {e}")
        # Optionally, return a default or error indicator result
        return None


In [10]:
# Import required libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import datasets
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ray import tune

# Assume X_train, X_test, y_train, y_test are pre-defined elsewhere in your script

# Function to configure and train models original
def train_and_evaluate(model_name, config):
    try:
        # Load tokenizer and model from Hugging Face model hub
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

        # Tokenization
        train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=config.get("max_length", 512))
        test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=config.get("max_length", 512))

        # Creation of datasets
        train_dataset = datasets.Dataset.from_dict({'input_ids': train_encodings['input_ids'], 'labels': y_train})
        test_dataset = datasets.Dataset.from_dict({'input_ids': test_encodings['input_ids'], 'labels': y_test})

        # Training configurations
        training_args = TrainingArguments(
            output_dir=f'./results/{model_name}',  # Output directory
            num_train_epochs=config["num_epochs"],  # Number of training epochs
            learning_rate=config['lr'],  # Learning rate
            per_device_train_batch_size=config['batch_size'],  # Batch size per device
            evaluation_strategy='epoch'  # Evaluate at the end of each epoch
        )

        # Trainer setup
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=test_dataset,
            compute_metrics=lambda p: {'accuracy': accuracy_score(p.label_ids, np.argmax(p.predictions, axis=1))}
        )

        # Training and evaluation
        trainer.train()
        eval_results = trainer.evaluate()

        # Reporting the evaluation loss and accuracy to Ray Tune
        tune.CLIReporter(metric_columns=[loss:=eval_results['eval_loss'], accuracy:=eval_results['eval_metrics']['accuracy']])
        

        # Confusion matrix visualization
        preds_output = trainer.predict(test_dataset)
        preds = np.argmax(preds_output.predictions, axis=-1)
        cm = confusion_matrix(y_test, preds)
        plt.figure(figsize=(7,5))
        sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=['Non-Fraud', 'Fraud'], yticklabels=['Non-Fraud', 'Fraud'])
        plt.xlabel('Predicted labels')
        plt.ylabel('True labels')
        plt.title(f'Confusion Matrix for {model_name}')
        plt.show()

        print(f"Model: {model_name}, Accuracy: {accuracy_score(y_test, preds)}")
        return eval_results['eval_loss']

    except Exception as e:
        print(f"Error training model {model_name}: {e}")
        # Report worst-case performance metrics to avoid trial hanging or crashing
        tune.CLIReporter(metric_columns=[loss:=float('inf'), accuracy:=0])
        #tune.CLIReporter(metric_columns=[loss=eval_results['eval_loss'], accuracy=eval_results['eval_metrics']['accuracy']])
        return float('inf')  # Return the worst possible loss in case of failure


In [11]:
# Configuração e execução da otimização de hiperparâmetros com Ray Tune
def optimize_and_train(model_name):
    def train_model(config):
        return train_and_evaluate(model_name, config)

    config = {
        "lr": tune.loguniform(1e-5, 5e-5),
        "batch_size": tune.choice([8, 16, 32]),
        "num_epochs": tune.choice([2, 3, 4]),
        "max_length": tune.choice([128, 256, 512])
    }

    scheduler = ASHAScheduler(
        metric="loss",  # or another metric like "accuracy"
        mode="min",     # use "max" if you need to maximize the metric
        max_t=10,
        grace_period=1,
        reduction_factor=2
    )
    result = tune.run(
        train_model,
        resources_per_trial={"cpu": 8, "gpu": 1},
        config=config,
        num_samples=10,
        scheduler=scheduler
    )

    # Acessar o melhor trial e seu último resultado
    print(result.get_best_result(metric="loss", mode="min").config)

    
#  best_trial = result.get_best_trial("loss", "min", "last")
#    if 'loss' in best_trial.last:
#        print("Best trial final validation loss: ", best_trial.last['loss'])
#        print("Best checkpoint path: ", best_trial.last["checkpoint"])
#    else:
#        print("Loss metric not found in the best trial results.")

In [12]:
# Carregar dados
X_train, X_test, y_train, y_test = load_data('Galerias.xlsx')

In [13]:
# Iniciar Ray
ray.init(ignore_reinit_error=True)

Usage stats collection is enabled by default for nightly wheels. To disable this, run the following command: `ray disable-usage-stats` before starting Ray. See https://docs.ray.io/en/master/cluster/usage-stats.html for more details.


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf8 in position 0: invalid start byte

In [ ]:
# Lista de modelos a serem avaliados
models = ['bert-base-uncased', 'roberta-base', 'google/electra-small-discriminator']

In [ ]:
# Loop para treinar e otimizar cada modelo
for model in models:
    print(f"Training and optimizing {model}")
    optimize_and_train(model)

In [ ]:
# Finalizar Ray
ray.shutdown()

In [ ]:
!dpkg -l | grep TensorRT

In [ ]:
import tensorflow as tf

# Verifica a disponibilidade de GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Configura o TensorFlow para usar apenas a primeira GPU
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("GPUs available:", gpus)
    except RuntimeError as e:
        print(e)
else:
    print("No GPU available.")

# Testa uma operação simples de TensorFlow na GPU
with tf.device('/GPU:0'):  # Força a execução na GPU
    random_matrix = tf.random.uniform([1000, 1000])
    result = tf.matmul(random_matrix, tf.transpose(random_matrix))
    print("Success: TensorFlow ran a matrix multiplication on GPU.")


In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit  # Inicializa CUDA automaticamente
from pycuda.compiler import SourceModule

def test_pycuda():
    mod = SourceModule("""
    __global__ void multiply_them(float *dest, float *a, float *b)
    {
        const int i = threadIdx.x;
        dest[i] = a[i] * b[i];
    }
    """)

    # Obtendo a função do módulo compilado
    multiply_them = mod.get_function("multiply_them")

    import numpy as np
    a = np.random.randn(400).astype(np.float32)
    b = np.random.randn(400).astype(np.float32)

    dest = np.zeros_like(a)

    # Chamando a função do kernel
    multiply_them(cuda.Out(dest), cuda.In(a), cuda.In(b), block=(400,1,1), grid=(1,1))

    print("Test completed: ", np.allclose(dest, a * b))

test_pycuda()


In [ ]:
import tensorflow as tf

# Verifica se o TensorRT está disponível no TensorFlow
try:
    from tensorflow.python.compiler.tensorrt import trt  # Verifica se o TensorRT está disponível
    print("TensorRT está disponível no TensorFlow.")
except ImportError:
    print("TensorRT não está disponível no TensorFlow. Verifique sua instalação.")

# Verifica se o TensorRT GPU está disponível
if tf.test.is_gpu_available():
    print("TensorRT GPU está disponível.")
else:
    print("TensorRT GPU não está disponível.")

# Exibe a versão do TensorFlow GPU
print("Versão do TensorFlow GPU:", tf.__version__)


!pip install transformers onnx onnx-tf
